# Лабораторная работа 1. Язык программирования Python.

## Задание 1
Дан массив вида [a,b,c,d,e,a,a,b,c]
<br>
    a) Преобразуйте его в словарь вида:
    <ul>
    <li>a: [0,5,6]</li>
    <li>b: [1,7]</li>
    <li>...</li>
    <li>e: [4]</li>
    </ul>
    <br>
    б) Сделайте это в одну строчку кода (разрешается с меньшей эффективностью)
    <br>
## Решение:

In [1]:
def array_to_dict(array: list) -> dict:
    return {array[i]: [item[0] for item in list(enumerate(array)) if item[1] == array[i]] for i in range(len(array))}

In [2]:
array_to_dict(['a', 'b', 'c', 'd', 'e', 'a', 'a', 'b', 'c'])

{'a': [0, 5, 6], 'b': [1, 7], 'c': [2, 8], 'd': [3], 'e': [4]}

## Задание 2
Метрика Жаккара (Jaccard similarity) - способ измерения сходства между множествами.
Вычисляется как $$ \frac{|A \cap B|}{|A \cup B|}, \\ $$ т.е. размер пересечения поделить на размер объединения множеств.
<br>Реализуйте вычисление метрики не используя методов intersection и union типа set и не создавая промежуточных множеств.
<br>
## Решение:

In [3]:
def jaccard_similarity(set1: set, set2: set) -> float:
    intersec_size = 0
    union_size = len(set1) + len(set2)
    for item in set1:
        if item in set2:
            intersec_size += 1
            union_size -= 1
    return intersec_size / union_size

In [4]:
jaccard_similarity({1, 2, 3}, {2, 4, 5})

0.2

## Задание 4
Дан следующий [json-файл](https://pastebin.com/raw/edzB88Px)<br>
Преобразуйте данные в CSV-файл вида:<br>
item, country, year, sales<br>
ITEM1, Russia, 2010, 23<br>
ITEM2, Russia, 2011, 40<br>
...<br>
ITEM2, Russia, 2010, 40<br>
...
<br>
## Решение:

In [5]:
import requests
import csv

In [6]:
def get_json_from_pastebin(url: str) -> list:
    return requests.get(url).json()

In [7]:
def json_to_csv(json_data: list) -> None:
    sales_csv = open("sales.csv", "w")
    csv_writer = csv.writer(sales_csv, delimiter=",")

    headers = ["item", "country", "year", "sales"]
    csv_writer.writerow(headers)

    for elem in json_data:
        content = list(elem.values())
        item = content[0]
        years_and_sales = content[1]

        for country in years_and_sales:
            years = list(reversed(years_and_sales[country].keys()))
            sales = list(reversed(years_and_sales[country].values()))

            while years or sales:
                row = [item, country, years.pop(), sales.pop()]
                csv_writer.writerow(row)
                row.clear()

    sales_csv.close()

In [8]:
json_data = get_json_from_pastebin("https://pastebin.com/raw/edzB88Px")
json_to_csv(json_data)

In [9]:
with open("sales.csv") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        print(row if row != [] else '\n', end='')

['item', 'country', 'year', 'sales']
['sweater', 'Russia', '2010', '85']
['sweater', 'Russia', '2011', '10']
['sweater', 'Russia', '2012', '104']
['sweater', 'Russia', '2013', '59']
['sweater', 'Russia', '2014', '113']
['sweater', 'Russia', '2015', '21']
['sweater', 'Russia', '2016', '23']
['sweater', 'Russia', '2017', '94']
['sweater', 'Russia', '2018', '134']
['sweater', 'Russia', '2019', '152']
['sweater', 'Belarus', '2010', '34']
['sweater', 'Belarus', '2011', '87']
['sweater', 'Belarus', '2012', '91']
['sweater', 'Belarus', '2013', '71']
['sweater', 'Belarus', '2014', '44']
['sweater', 'Belarus', '2015', '61']
['sweater', 'Belarus', '2016', '124']
['sweater', 'Belarus', '2017', '82']
['sweater', 'Belarus', '2018', '33']
['sweater', 'Belarus', '2019', '85']
['sweater', 'Ukraine', '2010', '53']
['sweater', 'Ukraine', '2011', '39']
['sweater', 'Ukraine', '2012', '32']
['sweater', 'Ukraine', '2013', '102']
['sweater', 'Ukraine', '2014', '121']
['sweater', 'Ukraine', '2015', '103']
['s

## Задание 6
Дан [граф друзей](https://raw.githubusercontent.com/Klemma/AIML_labs/master/arcs_refined.txt) из соцсети VK<br>
Необходимо:<br>
1) Определить число уникальных пользователей<br>
2) Для каждого id пользователя определить число его друзей из данного графа. Вывести топ 15. Учитывать неориентированность графа.<br>
3) Для всех возможных пар пользователей (декартова произведения):<br>
Для длины L 1..6 определить пропорцию пар пользователей, для которой кратчайший путь между пользователями в паре составляет L. 
Определить пропорцию, для которой этот путь больше 6.
Убедиться, что полученные пропорции суммируются в 1.
<br>
## Решение:
### Пункт 1

In [10]:
def get_txt_from_url(url: str) -> str:
    return requests.get(url).text

In [11]:
def get_users_set(txt: str) -> set:
    return set(txt.split())

In [12]:
txt = get_txt_from_url("https://raw.githubusercontent.com/Klemma/AIML_labs/master/arcs_refined.txt")
users = get_users_set(txt)
print(f"Количество уникальных пользователей равно {len(users)}")

Количество уникальных пользователей равно 3766


### Пункт 2

In [13]:
def get_users_friends_amount(txt: str) -> dict:
    txt_splitted = txt.split()
    users_friends_amount = dict()

    for item in txt_splitted:
        users_friends_amount[item] = 1 if item not in users_friends_amount else users_friends_amount[item] + 1

    return users_friends_amount

In [14]:
users_friends_amount = sorted(get_users_friends_amount(txt).items(), key=lambda x: x[1], reverse=True)
print("\nТоп 15 пользователей по количеству друзей:")
for i in range(15):
    print(users_friends_amount[i])


Топ 15 пользователей по количеству друзей:
('154572605', 351)
('136435809', 211)
('34847138', 210)
('1997236', 178)
('472893795', 178)
('270418723', 167)
('246825563', 155)
('120109289', 148)
('83372852', 146)
('94780691', 140)
('63962118', 136)
('150491520', 135)
('94509311', 135)
('122784747', 132)
('41459412', 130)


### Пункт 3

In [15]:
import numpy as np
from scipy.sparse.csgraph import dijkstra


def get_users_proportions(txt: str) -> dict:
    users = get_users_set(txt)
    users_enum = {val: key for key, val in dict(enumerate(users)).items()}

    users_pairs = [(users_enum[pair[0]], users_enum[pair[1]])
                   for pair in [pair.split(' ') for pair in txt.split('\n')]]

    adjacency_matrix = np.zeros((len(users), len(users)))
    for i in range(len(users_pairs)):
        user1 = users_pairs[i][0]
        user2 = users_pairs[i][1]
        adjacency_matrix[user1][user2] = 1
        adjacency_matrix[user2][user1] = 1

    dist_matrix = dijkstra(adjacency_matrix, directed=False)
    max_len = int(np.unique(dist_matrix)[-2])
    
    cartesian_product = [(user1, user2) for user1 in users for user2 in users if user1 != user2]
    
    proportion = lambda length: np.count_nonzero(dist_matrix == length) / len(cartesian_product)

    proportions = {length: proportion(length) for length in range(1, max_len + 1)}
    proportions[np.inf] = proportion(np.inf)
    
    return proportions

In [16]:
txt = get_txt_from_url("https://raw.githubusercontent.com/Klemma/AIML_labs/master/arcs_refined.txt")
proportions = get_users_proportions(txt)

<b>Полученные пропорции:</b>

In [17]:
proportions

{1: 0.0058403313635174295,
 2: 0.10119832230645483,
 3: 0.4391326885765488,
 4: 0.3318412665500152,
 5: 0.08070490211220968,
 6: 0.012568031996637278,
 7: 0.0023819750207877996,
 8: 0.0009966859416643923,
 9: 0.0004389593334927241,
 10: 8.674806879756598e-05,
 11: 7.899011142542592e-06,
 12: 2.821075408050926e-07,
 inf: 0.024801907611190926}

<b>Сумма полученных пропорций:</b>

In [18]:
sum(proportions.values())

1.0